In [23]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup

url = "https://en.wikipedia.org/wiki/Comparison_of_smartphones"

data = urlopen(url)
html = data.read()
data.close()
phone_soup = soup(html,'html.parser')
phones = phone_soup.findAll('table',{})

# phone[0] contain table of 2020 phones, phone[1] contain table of 2019 phones and so on.... till 2004 year

phone = phones[0]
col_title = phone.findAll('th')
row_data = phone.findAll('tr')[1:]
filename = 'phones 2020.csv'
with open(filename,'w',encoding='utf') as f:
    column_titles = [ct.text[:-1] for ct in col_title]
    header_string = ','.join(column_titles)
    header_string += "\n"
    f.write(header_string)
    prev_data = []
    for rd in range(0,len(col_title)):
        prev_data.append({"count" : 0,"value" : "", "columnspan" : 0})
    for row in row_data :
        data = row.findAll('td', {})
        str_data = []
        j = 0
        totalcol = 0
        i = 0
        while i<len(col_title):
            prev = int(prev_data[i]["count"])
            if prev > 0 :
                val = prev_data[i]["value"]
                str_data.append(val)
                count = int(prev_data[i]["columnspan"])
                prev_data[i] = {"count" : prev-1,"value" : val,"columnspan" : count}
                i+=count
                for k in range (0,count):
                    str_data.append(val)
            else :
                rd = data[j]
                val = rd.text
                val = val.replace(',','')
                val = val.replace('\n','')
                if(val == ""):
                    val = "-"
                if rd.get('rowspan') and rd.get('colspan'):
                    count = int(rd['rowspan'])-1
                    countcol = int(rd['colspan'])-1
                    prev_data[i] = {"count":count,"value":val,"columnspan":countcol}
                    totalcol +=count
                    i+=count
                    for k in range (0,count):
                        str_data.append(val)
                elif rd.get('rowspan'):
                    span = rd['rowspan']
                    if(span.isdigit()):
                        count = int(span)-1
                        prev_data[i] = {"count":count,"value":val,"columnspan":0}
                elif rd.get('colspan'):
                    count = int(rd['colspan'])-1
                    totalcol+=count
                    i+=count
                    for k in range (0,count):
                        str_data.append(val)
                    prev_data[i] = {"count":0,"value":val,"columnspan":count}
                else:
                    prev_data[i] = {"count":0,"value":val,"columnspan":0}
                str_data.append(val)
                j+=1
            i+=1
        row_string = ','.join(str_data)
        row_string += '\n'
        f.write(row_string)

In [26]:
import pandas as pd
df = pd.read_csv('phones 2020.csv')
df.head()

,Model,CPU,CPU specifications,GPU,Storage capacity,Removable storage,RAM,OS,Custom launcher,Dimensions,Weight,Battery,Recharge power,Display,Camera,Fingerprint scanner,Facial recognition
0,Samsung Galaxy S10 Lite,Snapdragon 855,Octa-core 2.84 GHz (1x 2.84 GHz + 3x 2.42 GHz ...,Adreno 640,128 GB,microSD up to 1 TB,6/8 GB,Android 10,One UI,162.5 x 75.6 x 8.1 mm,186g,4500 mAh,USB-C,"6.7"" 2400x1080 AMOLED",Triple 48 MP + 12 MP + 5 MP (rear) 32 MP (front),Optical,Yes
1,Samsung Galaxy Note 10 Lite,Exynos 9810,Octa-core 2.7 GHz (4x 2.7 GHz Mongoose M3 + 4x...,Mali-G72 MP18,128 GB,microSD up to 1 TB,6/8 GB,Android 10,One UI,163.7 x 76.1 x 8.7 mm,199g,4500 mAh,USB-C,"6.7"" 2400x1080 AMOLED",Triple 12 MP (rear) 32 MP (front),Optical,Yes
2,Samsung Galaxy Xcover Pro,Exynos 9611,Octa-core 2.3 GHz (4x 2.3 GHz Cortex-A73 + 4x ...,Mali-G72 MP3,64 GB,microSD up to 512 GB,4 GB,Android 10,One UI,159.9 x 76.7 x 10 mm,179g,4050mAh,USB-C 15W,"6.3"" 2340x1080 IPS LCD",Dual 25 MP + 8 MP (rear) 13 MP (front),Side-mounted,Yes
3,ZTE Axon 10s Pro 5G,Snapdragon 865,Octa-core 2.84 GHz (1x 2.84 GHz + 3x 2.42 GHz ...,Adreno 650,128/256 GB,microSDXC,6/12 GB,Android 10,MiFavor 10,159.2 x 73.4 x 7.9 mm,175g,4000mAh,USB-C 18W; Wireless,"6.5"" 2340x1080 AMOLED",Triple 48 MP + 20 MP + 8 MP (rear) 20 MP (front),Optical,Yes
4,Realme X50 5G,Snapdragon 765,Octa-core (1x 2.4 GHz Kryo 475 Prime + 1x 2.2 ...,Adreno 620,128/256 GB,None,6/8/12 GB,Android 10,ColorOS 7,163.8 x 75.8 x 8.9 mm,202g,4200 mAh,USB-C 30W,"6.6"" 2400x1080 120Hz IPS LCD",Quad 64 MP + 12 MP + 8 MP + 2 MP (rear) 16 MP ...,Side-mounted,Yes
